## 수정 필요
1. validation 후 best parameter 적용한 모델로 test dataset 분석
2. test시에도 train data는 validation data 제외하도록 수정

In [8]:
# 한글 폰트 설치
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 40 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 1s (9,734 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 155501 files and d

In [1]:
pip install gtts

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\anaconda3\envs\cakd3\python.exe -m pip install --upgrade pip' command.


In [1]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
from gtts import gTTS
import os 
from IPython.display import Audio
import warnings
warnings.filterwarnings('ignore')

def speak(text):
    tts = gTTS(text=text, lang='ko') 
    filename='voice.mp3' 
    tts.save(filename)
    display(Audio(filename, autoplay=True))
    os.remove(filename)

In [2]:
# change directory
%cd /content/drive/MyDrive/Cakd3_Project/1.ldata_현정

/content/drive/MyDrive/Cakd3_Project/1.ldata_현정


In [3]:
# 데이터셋 불러오기
dataset1 = pd.read_csv('./full_dataset1.csv', index_col=0)
dataset2 = pd.read_csv('./full_dataset2.csv', index_col=0)

In [14]:
# classification

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier, plot_importance
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import warnings
warnings.filterwarnings('ignore')


def clf(train, test=None):
    
    if test is not None:
        mode = '테스트'
        X_train = train.drop('y', axis=1)
        y_train = train['y']

        X_test = test.drop('y', axis=1)
        y_test = test['y']
        
    else:
        mode = '검증'
        X = train.drop('y', axis=1)
        y = train['y']
        X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                           test_size=0.3,
                                                           random_state=1004)

    # 객체 생성
    dct_clf = DecisionTreeClassifier(criterion = 'entropy')
    rf_clf = RandomForestClassifier()
    lr_clf = LogisticRegression()
    xgb_clf = XGBClassifier(eval_metric='logloss')
    lgb_clf = LGBMClassifier()
    gb_clf = GradientBoostingClassifier()
    # svm_clf = SVC(probability=True)

    # 파라미터 설정
    dct_parameters = {'max_depth':[3,5], }
    rf_parameters = {'n_estimators':[100,200,300], 'max_depth':[7,9,11]}
    lr_parameters = { "penalty":['l1', 'l2', 'elasticnet', 'none'], 'C': [ 1, 10, 100, 1000]}
    xgb_parameters = {'n_estimators':[300], 'learning_rate':[0.05, 0.1], 'max_depth':[4,5,6]}
    lgb_parameters = {'n_estimators':[300], 'learning_rate':[0.05, 0.1], 'max_depth':[4,5,6]}

    clf_param = [(dct_clf,dct_parameters),(rf_clf,rf_parameters),(lr_clf,lr_parameters),
                 (xgb_clf,xgb_parameters),(lgb_clf,lgb_parameters)]
    
    for clf, parameter in clf_param:
        grid_clf = GridSearchCV(clf, param_grid=parameter, scoring='accuracy', cv=3, refit=True)
        grid_clf.fit(X_train, y_train)

        # 교차검증 결과 출력
        class_name = clf.__class__.__name__
        scores_df = pd.DataFrame(grid_clf.cv_results_)
        # display(HTML(scores_df[['params','rank_test_score','mean_test_score']].to_html()))
        print(f'{class_name} 최적 하이퍼 파라미터:', grid_clf.best_params_)
        print('{0} 최고 정확도:{1:.4f}'.format(class_name,grid_clf.best_score_))

        # x_test에 최적 하이퍼 파라미터 적용하여 분석한 결과
        best_clf = grid_clf.best_estimator_
        pred = best_clf.predict(X_test)
        pred_proba = best_clf.predict_proba(X_test)[:,1]

        accuracy = accuracy_score(y_test, pred)
        precision = precision_score(y_test, pred)
        recall = recall_score(y_test, pred)
        f1 = f1_score(y_test, pred)
        auc = roc_auc_score(y_test, pred_proba)

        print(f'< {mode} 데이터셋 >')
        print('accuracy  :  {:.4f}'.format(accuracy))
        print('precision :  {:.4f}'.format(precision))
        print('recall    :  {:.4f}'.format(recall))
        print('f1        :  {:.4f}'.format(f1))
        print('roc_auc   :  {:.4f}\n'.format(auc))

# full dataset

In [23]:
# 검증 데이터셋
clf(dataset1)

DecisionTreeClassifier 최적 하이퍼 파라미터: {'max_depth': 5}
DecisionTreeClassifier 최고 정확도:0.6865
< 검증 데이터셋 >
accuracy  :  0.6930
precision :  0.6345
recall    :  0.6271
f1        :  0.6308
roc_auc   :  0.7559

RandomForestClassifier 최적 하이퍼 파라미터: {'max_depth': 7, 'n_estimators': 300}
RandomForestClassifier 최고 정확도:0.7124
< 검증 데이터셋 >
accuracy  :  0.7117
precision :  0.6697
recall    :  0.6134
f1        :  0.6403
roc_auc   :  0.7811

LogisticRegression 최적 하이퍼 파라미터: {'C': 1, 'penalty': 'l2'}
LogisticRegression 최고 정확도:0.6496
< 검증 데이터셋 >
accuracy  :  0.6545
precision :  0.6046
recall    :  0.5027
f1        :  0.5490
roc_auc   :  0.7021

XGBClassifier 최적 하이퍼 파라미터: {'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 300}
XGBClassifier 최고 정확도:0.7088
< 검증 데이터셋 >
accuracy  :  0.7105
precision :  0.6600
recall    :  0.6350
f1        :  0.6473
roc_auc   :  0.7815

LGBMClassifier 최적 하이퍼 파라미터: {'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 300}
LGBMClassifier 최고 정확도:0.7059
< 검증 데이터셋 >
accuracy  

In [24]:
speak('검증 데이터셋 완료')

In [25]:
# 테스트 데이터셋
clf(dataset1, dataset2)

DecisionTreeClassifier 최적 하이퍼 파라미터: {'max_depth': 5}
DecisionTreeClassifier 최고 정확도:0.6965
< 테스트 데이터셋 >
accuracy  :  0.6834
precision :  0.6860
recall    :  0.6503
f1        :  0.6677
roc_auc   :  0.7454

RandomForestClassifier 최적 하이퍼 파라미터: {'max_depth': 9, 'n_estimators': 300}
RandomForestClassifier 최고 정확도:0.7124
< 테스트 데이터셋 >
accuracy  :  0.6941
precision :  0.7011
recall    :  0.6529
f1        :  0.6761
roc_auc   :  0.7628

LogisticRegression 최적 하이퍼 파라미터: {'C': 10, 'penalty': 'l2'}
LogisticRegression 최고 정확도:0.6506
< 테스트 데이터셋 >
accuracy  :  0.6306
precision :  0.6551
recall    :  0.5169
f1        :  0.5778
roc_auc   :  0.6924

XGBClassifier 최적 하이퍼 파라미터: {'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 300}
XGBClassifier 최고 정확도:0.7062
< 테스트 데이터셋 >
accuracy  :  0.6918
precision :  0.6919
recall    :  0.6665
f1        :  0.6790
roc_auc   :  0.7623

LGBMClassifier 최적 하이퍼 파라미터: {'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 300}
LGBMClassifier 최고 정확도:0.7073
< 테스트 데이터셋 >
accu

In [26]:
speak('테스트 데이터셋 완료')

# small dataset

In [17]:
# 데이터셋 불러오기
s_dataset1 = pd.read_csv('./small_dataset1.csv', index_col=0)
s_dataset2 = pd.read_csv('./small_dataset2.csv', index_col=0)

In [19]:
# 검증 데이터셋
clf(s_dataset1)

DecisionTreeClassifier 최적 하이퍼 파라미터: {'max_depth': 3}
DecisionTreeClassifier 최고 정확도:0.7076
< 검증 데이터셋 >
accuracy  :  0.7175
precision :  0.6678
recall    :  0.6459
f1        :  0.6567
roc_auc   :  0.7705

RandomForestClassifier 최적 하이퍼 파라미터: {'max_depth': 7, 'n_estimators': 200}
RandomForestClassifier 최고 정확도:0.7155
< 검증 데이터셋 >
accuracy  :  0.7180
precision :  0.6694
recall    :  0.6438
f1        :  0.6563
roc_auc   :  0.7860

LogisticRegression 최적 하이퍼 파라미터: {'C': 1, 'penalty': 'l2'}
LogisticRegression 최고 정확도:0.7136
< 검증 데이터셋 >
accuracy  :  0.7140
precision :  0.6707
recall    :  0.6213
f1        :  0.6451
roc_auc   :  0.7759

XGBClassifier 최적 하이퍼 파라미터: {'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 300}
XGBClassifier 최고 정확도:0.7095
< 검증 데이터셋 >
accuracy  :  0.7104
precision :  0.6565
recall    :  0.6450
f1        :  0.6507
roc_auc   :  0.7840

LGBMClassifier 최적 하이퍼 파라미터: {'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 300}
LGBMClassifier 최고 정확도:0.7063
< 검증 데이터셋 >
accuracy  

In [20]:
speak('검증 데이터셋 완료')

In [21]:
# 테스트 데이터셋
clf(s_dataset1, s_dataset2)

DecisionTreeClassifier 최적 하이퍼 파라미터: {'max_depth': 3}
DecisionTreeClassifier 최고 정확도:0.7085
< 테스트 데이터셋 >
accuracy  :  0.6956
precision :  0.7228
recall    :  0.6125
f1        :  0.6631
roc_auc   :  0.7586

RandomForestClassifier 최적 하이퍼 파라미터: {'max_depth': 9, 'n_estimators': 100}
RandomForestClassifier 최고 정확도:0.7142
< 테스트 데이터셋 >
accuracy  :  0.6992
precision :  0.7019
recall    :  0.6692
f1        :  0.6851
roc_auc   :  0.7674

LogisticRegression 최적 하이퍼 파라미터: {'C': 1, 'penalty': 'l2'}
LogisticRegression 최고 정확도:0.7094
< 테스트 데이터셋 >
accuracy  :  0.6861
precision :  0.7005
recall    :  0.6255
f1        :  0.6609
roc_auc   :  0.7567

XGBClassifier 최적 하이퍼 파라미터: {'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 300}
XGBClassifier 최고 정확도:0.7090
< 테스트 데이터셋 >
accuracy  :  0.7001
precision :  0.7001
recall    :  0.6764
f1        :  0.6881
roc_auc   :  0.7685

LGBMClassifier 최적 하이퍼 파라미터: {'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 300}
LGBMClassifier 최고 정확도:0.7087
< 테스트 데이터셋 >
accur

In [22]:
speak('테스트 데이터셋 완료')

# sfm dataset

In [33]:
# 데이터셋 불러오기
sfm_dataset1 = pd.read_csv('./sfm_dataset1.csv', index_col=0)
sfm_dataset2 = pd.read_csv('./sfm_dataset2.csv', index_col=0)

# 검증 데이터셋
clf(sfm_dataset1)

speak('검증 데이터셋 완료')

DecisionTreeClassifier 최적 하이퍼 파라미터: {'max_depth': 5}
DecisionTreeClassifier 최고 정확도:0.6929
< 검증 데이터셋 >
accuracy  :  0.6950
precision :  0.6400
recall    :  0.6192
f1        :  0.6294
roc_auc   :  0.7573

RandomForestClassifier 최적 하이퍼 파라미터: {'max_depth': 11, 'n_estimators': 300}
RandomForestClassifier 최고 정확도:0.7115
< 검증 데이터셋 >
accuracy  :  0.7117
precision :  0.6636
recall    :  0.6305
f1        :  0.6466
roc_auc   :  0.7833

LogisticRegression 최적 하이퍼 파라미터: {'C': 1, 'penalty': 'none'}
LogisticRegression 최고 정확도:0.6242
< 검증 데이터셋 >
accuracy  :  0.6277
precision :  0.5981
recall    :  0.3350
f1        :  0.4294
roc_auc   :  0.6751

XGBClassifier 최적 하이퍼 파라미터: {'learning_rate': 0.05, 'max_depth': 6, 'n_estimators': 300}
XGBClassifier 최고 정확도:0.7082
< 검증 데이터셋 >
accuracy  :  0.7107
precision :  0.6619
recall    :  0.6305
f1        :  0.6458
roc_auc   :  0.7745

LGBMClassifier 최적 하이퍼 파라미터: {'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 300}
LGBMClassifier 최고 정확도:0.7072
< 검증 데이터셋 >
accurac

In [34]:
# 테스트 데이터셋
clf(sfm_dataset1, sfm_dataset2)

speak('테스트 데이터셋 완료')

DecisionTreeClassifier 최적 하이퍼 파라미터: {'max_depth': 5}
DecisionTreeClassifier 최고 정확도:0.6990
< 테스트 데이터셋 >
accuracy  :  0.6820
precision :  0.6818
recall    :  0.6561
f1        :  0.6687
roc_auc   :  0.7419

RandomForestClassifier 최적 하이퍼 파라미터: {'max_depth': 11, 'n_estimators': 300}
RandomForestClassifier 최고 정확도:0.7109
< 테스트 데이터셋 >
accuracy  :  0.6960
precision :  0.7005
recall    :  0.6610
f1        :  0.6802
roc_auc   :  0.7627

LogisticRegression 최적 하이퍼 파라미터: {'C': 1, 'penalty': 'none'}
LogisticRegression 최고 정확도:0.6173
< 테스트 데이터셋 >
accuracy  :  0.5749
precision :  0.6408
recall    :  0.2974
f1        :  0.4063
roc_auc   :  0.6540

XGBClassifier 최적 하이퍼 파라미터: {'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 300}
XGBClassifier 최고 정확도:0.7067
< 테스트 데이터셋 >
accuracy  :  0.6932
precision :  0.6942
recall    :  0.6662
f1        :  0.6799
roc_auc   :  0.7608

LGBMClassifier 최적 하이퍼 파라미터: {'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 300}
LGBMClassifier 최고 정확도:0.7073
< 테스트 데이터셋 >
ac

# 전진선택법 dataset

In [35]:
# 데이터셋 불러오기
f_dataset1 = pd.read_csv('./f_dataset1.csv', index_col=0)
f_dataset2 = pd.read_csv('./f_dataset2.csv', index_col=0)

# 검증 데이터셋
clf(f_dataset1)

speak('검증 데이터셋 완료')

DecisionTreeClassifier 최적 하이퍼 파라미터: {'max_depth': 5}
DecisionTreeClassifier 최고 정확도:0.6939
< 검증 데이터셋 >
accuracy  :  0.6942
precision :  0.6407
recall    :  0.6122
f1        :  0.6261
roc_auc   :  0.7571

RandomForestClassifier 최적 하이퍼 파라미터: {'max_depth': 11, 'n_estimators': 100}
RandomForestClassifier 최고 정확도:0.7128
< 검증 데이터셋 >
accuracy  :  0.7077
precision :  0.6581
recall    :  0.6271
f1        :  0.6422
roc_auc   :  0.7797

LogisticRegression 최적 하이퍼 파라미터: {'C': 1, 'penalty': 'none'}
LogisticRegression 최고 정확도:0.6130
< 검증 데이터셋 >
accuracy  :  0.6073
precision :  0.5632
recall    :  0.2726
f1        :  0.3674
roc_auc   :  0.6422

XGBClassifier 최적 하이퍼 파라미터: {'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 300}
XGBClassifier 최고 정확도:0.7077
< 검증 데이터셋 >
accuracy  :  0.7081
precision :  0.6570
recall    :  0.6321
f1        :  0.6443
roc_auc   :  0.7787

LGBMClassifier 최적 하이퍼 파라미터: {'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 300}
LGBMClassifier 최고 정확도:0.7082
< 검증 데이터셋 >
accurac

In [36]:
# 테스트 데이터셋
clf(f_dataset1, f_dataset2)

speak('테스트 데이터셋 완료')

DecisionTreeClassifier 최적 하이퍼 파라미터: {'max_depth': 5}
DecisionTreeClassifier 최고 정확도:0.6989
< 테스트 데이터셋 >
accuracy  :  0.6808
precision :  0.6819
recall    :  0.6512
f1        :  0.6662
roc_auc   :  0.7439

RandomForestClassifier 최적 하이퍼 파라미터: {'max_depth': 11, 'n_estimators': 300}
RandomForestClassifier 최고 정확도:0.7101
< 테스트 데이터셋 >
accuracy  :  0.6931
precision :  0.6970
recall    :  0.6587
f1        :  0.6773
roc_auc   :  0.7633

LogisticRegression 최적 하이퍼 파라미터: {'C': 1, 'penalty': 'none'}
LogisticRegression 최고 정확도:0.6077
< 테스트 데이터셋 >
accuracy  :  0.5653
precision :  0.6265
recall    :  0.2750
f1        :  0.3822
roc_auc   :  0.6329

XGBClassifier 최적 하이퍼 파라미터: {'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 300}
XGBClassifier 최고 정확도:0.7066
< 테스트 데이터셋 >
accuracy  :  0.6927
precision :  0.6941
recall    :  0.6646
f1        :  0.6790
roc_auc   :  0.7612

LGBMClassifier 최적 하이퍼 파라미터: {'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 300}
LGBMClassifier 최고 정확도:0.7076
< 테스트 데이터셋 >
ac

# 후진선택법 dataset

In [37]:
# 데이터셋 불러오기
b_dataset1 = pd.read_csv('./b_dataset1.csv', index_col=0)
b_dataset2 = pd.read_csv('./b_dataset2.csv', index_col=0)

# 검증 데이터셋
clf(b_dataset1)

speak('검증 데이터셋 완료')

DecisionTreeClassifier 최적 하이퍼 파라미터: {'max_depth': 5}
DecisionTreeClassifier 최고 정확도:0.6933
< 검증 데이터셋 >
accuracy  :  0.6982
precision :  0.6450
recall    :  0.6192
f1        :  0.6318
roc_auc   :  0.7566

RandomForestClassifier 최적 하이퍼 파라미터: {'max_depth': 9, 'n_estimators': 200}
RandomForestClassifier 최고 정확도:0.7135
< 검증 데이터셋 >
accuracy  :  0.7142
precision :  0.6683
recall    :  0.6288
f1        :  0.6479
roc_auc   :  0.7818

LogisticRegression 최적 하이퍼 파라미터: {'C': 1000, 'penalty': 'l2'}
LogisticRegression 최고 정확도:0.6169
< 검증 데이터셋 >
accuracy  :  0.6228
precision :  0.5851
recall    :  0.3375
f1        :  0.4281
roc_auc   :  0.6553

XGBClassifier 최적 하이퍼 파라미터: {'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 300}
XGBClassifier 최고 정확도:0.7083
< 검증 데이터셋 >
accuracy  :  0.7044
precision :  0.6552
recall    :  0.6192
f1        :  0.6367
roc_auc   :  0.7781

LGBMClassifier 최적 하이퍼 파라미터: {'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 300}
LGBMClassifier 최고 정확도:0.7078
< 검증 데이터셋 >
accurac

In [38]:
# 테스트 데이터셋
clf(b_dataset1, b_dataset2)

speak('테스트 데이터셋 완료')

DecisionTreeClassifier 최적 하이퍼 파라미터: {'max_depth': 5}
DecisionTreeClassifier 최고 정확도:0.6988
< 테스트 데이터셋 >
accuracy  :  0.6808
precision :  0.6819
recall    :  0.6512
f1        :  0.6662
roc_auc   :  0.7433

RandomForestClassifier 최적 하이퍼 파라미터: {'max_depth': 9, 'n_estimators': 100}
RandomForestClassifier 최고 정확도:0.7101
< 테스트 데이터셋 >
accuracy  :  0.6918
precision :  0.6973
recall    :  0.6535
f1        :  0.6747
roc_auc   :  0.7623

LogisticRegression 최적 하이퍼 파라미터: {'C': 1, 'penalty': 'none'}
LogisticRegression 최고 정확도:0.6121
< 테스트 데이터셋 >
accuracy  :  0.5725
precision :  0.6283
recall    :  0.3081
f1        :  0.4134
roc_auc   :  0.6409

XGBClassifier 최적 하이퍼 파라미터: {'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 300}
XGBClassifier 최고 정확도:0.7075
< 테스트 데이터셋 >
accuracy  :  0.6914
precision :  0.6932
recall    :  0.6621
f1        :  0.6773
roc_auc   :  0.7612

LGBMClassifier 최적 하이퍼 파라미터: {'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 300}
LGBMClassifier 최고 정확도:0.7054
< 테스트 데이터셋 >
acc

In [39]:
speak('돌아오시오~~')

# <font color='green'>변수 중요도(XGB)</font>

In [ ]:
# # 변수 중요도(XGB)
# import matplotlib.pyplot as plt
# %matplotlib inline

# xgb_clf = XGBClassifier(learning_rate=0.05, max_depth=4, n_estimators=300)
# xgb_clf.fit(X_train, y_train)
# fig, ax = plt.subplots(1,1,figsize=(10,8))
# plot_importance(xgb_clf, ax=ax, max_num_features=20, height=0.4)